In [3]:
import numpy as np
import scipy as sp

import scipy
from scipy.stats import multivariate_normal
from scipy       import optimize

import skimage

import matplotlib.pyplot as plt

import pylab

from glob import glob
import sys
import os

import itk
from itkwidgets import view

from tqdm import tqdm

import math

from glob import glob

import matplotlib.pyplot as plt

import itk

import av
import ffmpeg

import json

import site
site.addsitedir('../../ARGUS')
from ARGUSUtils_IO import *
from ARGUSUtils_Linearization import *

In [8]:
def insert_overlays_into_volume(pathname, pluClass):
    data_overlay_paths = sorted(glob(pathname+"-Annotations-Linear/*.overlay.mha"))
    data_video_dir = pathname+"-Linear/"
    
    num_overlays = len(data_overlay_paths)
    
    i=0
    (next_path,next_filename) = os.path.split(data_overlay_paths[i])
    next_video_name = data_video_dir+next_filename.split('.')[0]+'.mha'
    while i<num_overlays:
        print(next_video_name)
        video_name = next_video_name
        filename = next_filename
        vidimg = itk.imread(video_name, itk.F)
        nbs = 0
        while np.mean(vidimg[nbs])<=1:
            nbs += 1
        print("   Video Blank: 0 -", nbs)
        for j in range(0,nbs):
            vidimg[j] = vidimg[nbs]
        nbs = vidimg.shape[0]-1
        while np.mean(vidimg[nbs])<=1:
            nbs -= 1
        print("   Video Blank: ", nbs+1, "-", vidimg.shape[0])
        for j in range(nbs+1,vidimg.shape[0]):
            vidimg[j] = vidimg[nbs]
        new_video_name = pathname+"-Annotations-Linear/"+filename.split('.')[0]+'.mha'
        itk.imwrite(vidimg, new_video_name)
        
        vidimg.FillBuffer(0)
        print("Processing", video_name)
        while next_video_name==video_name and i<num_overlays:
            filename = next_filename
            frame_num = int(filename.split('.')[1].split('-')[2][1:6])
            print("   ", frame_num)
            ovimg = itk.imread(data_overlay_paths[i], itk.UC)
            vidimg[frame_num,:,:] = ovimg
            i += 1
            if i<num_overlays:
                (next_path,next_filename) = os.path.split(data_overlay_paths[i])
                next_video_name = data_video_dir+next_filename.split('.')[0]+'.mha'
        new_video_name = pathname+"-Annotations-Linear/"+filename.split('.')[0]+'.inserted-overlay.mha'
        print("Writing", new_video_name)
        itk.imwrite(vidimg, new_video_name)
        vid = itk.GetArrayFromImage(vidimg)
        rib = np.where(vid == 2, 2, 0)
        interpRib = itk.MorphologicalContourInterpolator.New(itk.GetImageFromArray(rib.astype(np.ubyte)))
        interpRib.Update()
        rib = itk.GetArrayFromImage(interpRib.GetOutput())
        plu = np.where(vid == 1, pluClass, 0)
        interpPlu = itk.MorphologicalContourInterpolator.New(itk.GetImageFromArray(plu.astype(np.ubyte)))
        interpPlu.Update()
        plu = itk.GetArrayFromImage(interpPlu.GetOutput())
        label = np.maximum(rib,plu,dtype=np.ubyte)
        nbs = 0
        while np.sum(label[nbs])==0:
            nbs += 1
        print("   Blank: 0 -", nbs)
        for j in range(0,nbs):
            label[j] = label[nbs]
        nbs = label.shape[0]-1
        while np.sum(label[nbs])==0:
            nbs -= 1
        print("   Blank: ", nbs-1, "-", label.shape[0])
        for j in range(nbs+1,label.shape[0]):
            label[j] = label[nbs]
        vid = itk.GetImageFromArray(label)
        new_video_name = pathname+"-Annotations-Linear/"+filename.split('.')[0]+'.interpolated-overlay.mha'
        print("Writing", new_video_name)
        itk.imwrite(vid, new_video_name)

In [10]:
pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding"
insert_overlays_into_volume(pathname, 1)
pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXSliding"
insert_overlays_into_volume(pathname, 3)

../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Linear/Image_262499828648_clean.mha
   Video Blank: 0 - 0
   Video Blank:  68 - 68
Processing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Linear/Image_262499828648_clean.mha
    0
    4
    6
    11
    17
    24
    45
    48
    56
    63
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations-Linear/Image_262499828648_clean.inserted-overlay.mha
   Blank: 0 - 0
   Blank:  62 - 68
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations-Linear/Image_262499828648_clean.interpolated-overlay.mha
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Linear/image_1083297968960_clean.mha
   Video Blank: 0 - 0
   Video Blank:  61 - 61
Processing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Linear/image_1083297968960_clean.mha
    0
    6
    12
    19
    20
    34
    40
    52
    56
    60
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations-Linear/image_1083297968960_clean.inserted-overla

Processing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Linear/image_2734882394424_CLEAN.mha
    0
    10
    19
    20
    31
    34
    38
    40
    48
    58
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations-Linear/image_2734882394424_CLEAN.inserted-overlay.mha
   Blank: 0 - 0
   Blank:  57 - 61
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations-Linear/image_2734882394424_CLEAN.interpolated-overlay.mha
../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Linear/image_2743083265515_CLEAN.mha
   Video Blank: 0 - 0
   Video Blank:  61 - 61
Processing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Linear/image_2743083265515_CLEAN.mha
    0
    8
    11
    14
    19
    20
    28
    32
    34
    40
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations-Linear/image_2743083265515_CLEAN.inserted-overlay.mha
   Blank: 0 - 0
   Blank:  39 - 61
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding-Annotations-Linear/image_2743

Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_10391571128899_CLEAN.inserted-overlay.mha
   Blank: 0 - 0
   Blank:  59 - 61
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_10391571128899_CLEAN.interpolated-overlay.mha
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Linear/image_10395655826502_CLEAN.mha
   Video Blank: 0 - 0
   Video Blank:  60 - 60
Processing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Linear/image_10395655826502_CLEAN.mha
    0
    6
    16
    20
    31
    36
    40
    49
    54
    59
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_10395655826502_CLEAN.inserted-overlay.mha
   Blank: 0 - 0
   Blank:  58 - 60
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_10395655826502_CLEAN.interpolated-overlay.mha
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Linear/image_104543812690743_CLEAN.mha
   Video Blank: 0 - 0
   Video Blank:  61 - 6

   Blank: 0 - 0
   Blank:  39 - 61
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_1511338287338_clean.interpolated-overlay.mha
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Linear/image_1896534330004_clean.mha
   Video Blank: 0 - 0
   Video Blank:  60 - 60
Processing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Linear/image_1896534330004_clean.mha
    0
    4
    6
    10
    15
    20
    39
    40
    53
    58
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_1896534330004_clean.inserted-overlay.mha
   Blank: 0 - 0
   Blank:  57 - 60
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_1896534330004_clean.interpolated-overlay.mha
../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Linear/image_1901852337971_clean.mha
   Video Blank: 0 - 0
   Video Blank:  61 - 61
Processing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Linear/image_1901852337971_clean.mha
    0
    5
    9
    12
    15
 

Processing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Linear/image_74132233134844_clean.mha
    0
    8
    14
    19
    20
    29
    35
    39
    40
    54
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_74132233134844_clean.inserted-overlay.mha
   Blank: 0 - 0
   Blank:  53 - 60
Writing ../../Data/TrainingData/AR-UNet/BAMC-PTXSliding-Annotations-Linear/image_74132233134844_clean.interpolated-overlay.mha


In [11]:
pathname = "../../Data/TestingData/BAMC-PTXNoSliding"
insert_overlays_into_volume(pathname, 1)
pathname = "../../Data/TestingData/BAMC-PTXSliding"
insert_overlays_into_volume(pathname, 3)

../../Data/TestingData/BAMC-PTXNoSliding-Linear/image_417221672548_CLEAN.mha
   Video Blank: 0 - 0
   Video Blank:  61 - 61
Processing ../../Data/TestingData/BAMC-PTXNoSliding-Linear/image_417221672548_CLEAN.mha
    0
    5
    10
    17
    20
    26
    33
    40
    57
    60
Writing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear/image_417221672548_CLEAN.inserted-overlay.mha
   Blank: 0 - 0
   Blank:  59 - 61
Writing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear/image_417221672548_CLEAN.interpolated-overlay.mha
../../Data/TestingData/BAMC-PTXNoSliding-Linear/image_426794579576_CLEAN.mha
   Video Blank: 0 - 0
   Video Blank:  60 - 60
Processing ../../Data/TestingData/BAMC-PTXNoSliding-Linear/image_426794579576_CLEAN.mha
    0
    4
    7
    11
    14
    19
    20
    33
    37
    40
Writing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear/image_426794579576_CLEAN.inserted-overlay.mha
   Blank: 0 - 0
   Blank:  39 - 60
Writing ../../Data/Testin